# Significance Testing

In [1]:
import xarray as xr
import numpy as np
import scipy.signal as signal
import scipy.stats as stats
import esmtools.stats as esmstats

import matplotlib.pyplot as plt

In [2]:
def open_metric(var, reg, metric, timescale='monthly', ens_type=''):
    
    writedir = '/home/bbuchovecky/storage/so_predict_derived/'
    
    if metric == 'clim':
        subdir = 'CTRL/'+var.upper()+'/'
        filename = var.lower()+'_ts_'+reg+'_'+metric+'.nc'
    
    if metric == 'anom' or metric == 'mean' or (metric == 'var' and timescale == 'monthly'):
        subdir = 'CTRL/'+var.upper()+'/'
        filename = var.lower()+'_ts_'+reg+'_'+timescale+'_'+metric+'.nc'
    
    if metric.lower() == 'ppp':
        subdir = 'PPP/'+var.upper()+'/'
        if ens_type != '':
            ens_type += '_'
        filename = var.lower()+'_ts_'+reg+'_'+timescale+'_'+ens_type+'ppp.nc'
        
    return xr.open_dataset(writedir+subdir+filename)

def get_plotting_labels():
    with open('/home/bbuchovecky/storage/so_predict_derived/plotting_dicts.pkl','rb') as handle:
        plotting_dicts = pkl.load(handle)
    
    reg_names = plotting_dicts['reg_names']
    var_abbrv_names = plotting_dicts['var_abbrv_names']
    abbrv_month_names = plotting_dicts['abbrv_month_names']
    month_letters = plotting_dicts['month_letters']
    
    return reg_names, var_abbrv_names, abbrv_month_names, month_letters

## PM Experiment

We use an $F$-test, so we first need to estimate the effective degrees of freedom.

### PM ensemble
6 start years ($M=6$) each with 40 ensemble members ($N=40$)

$\text{DOF}=M\times N-1=6\times 40 -1=239$

### Control run
300 years, equivalent to 300 samples per month, but we need to account for autocorrelation using [Bretherton et al. (1999)](https://doi-org.ezproxy.princeton.edu/10.1175/1520-0442(1999)012<1990:TENOSD>2.0.CO;2).

> Different ‘‘realizations’’ would now correspond to independent sets of $T$ observations.

> Following the approach of section 4b of looking at the distribution of the sample covariance between two unrelated time series, one can derive an analogous formula for the ESS appropriate for significance tests of the correlation between two times series $X_i$ and $Y_i$ with different autocorrelation sequences $\rho_\tau^X$ and $\rho_\tau^Y$:
$$
T_{XY}^*=\frac{T}{\sum\limits_{\tau=-(T-1)}^{(T-1)} (1-|\tau|/T)\rho_\tau^X\rho_\tau^Y}
$$

For the control run, we are accounting for autocorrelation so the equation from Bretherton et al. (1999) becomes the following:
$$
T_{XX}^*=\frac{T}{\sum\limits_{\tau=-(T-1)}^{(T-1)} (1-|\tau|/T)\left(\rho_\tau^X\right)^2}
$$
Each month (300 samples) will have its own $T^*$ value, but we will simplify this by choosing the most conservative value (lowest $T^*$).

### Function for computing $T^*$

In [3]:
def compute_Tstar(ts1, ts2, T=300, return_corr=False):
    assert len(ts1) == len(ts2) == T
    cumsum = 0
    
    corr_rho_1 = np.zeros(2*T-3)
    corr_rho_2 = np.zeros(2*T-3)
    
    ## compute demoninator --> tau = [-298,298] since stats.pearsonr needs arrays with size >=2 
#     for i,tau in enumerate(range(-T+2,T-2+1)):
    for i,tau in enumerate(range(-10,10+1)):
        coeff = 1 - (abs(tau)/T)
        
        if tau < 0:
            rho_1 = stats.pearsonr(ts1[:tau], ts1[-tau:])[0] ## the [0] index just grabs the Pearson corr coeff and ignores the p-value
            rho_2 = stats.pearsonr(ts2[:tau], ts2[-tau:])[0]
        if tau == 0:
            rho_1 = stats.pearsonr(ts1, ts1)[0]
            rho_2 = stats.pearsonr(ts2, ts2)[0]
        if tau > 0:
            rho_1 = stats.pearsonr(ts1[tau:], ts1[:-tau])[0]
            rho_2 = stats.pearsonr(ts2[tau:], ts2[:-tau])[0]
            
        corr_rho_1[i] = rho_1
        corr_rho_2[i] = rho_2
            
        cumsum += coeff * rho_1 * rho_2
    
    ## return floor T* and autocorrelation vectors
    if return_corr:
        return corr_rho_1, corr_rho_2, T // cumsum
    
    ## return floor T*
    if not return_corr:
        return T // cumsum

In [4]:
def compute_PM_ctrl_run_Tstar(var, return_arr=False):
    '''
    Returns
    -------
    Tstar : array-like
        An array of dim 12x6 with Tstar for each month and region [month x region]
    '''
    ts = open_metric(var,'so','anom')
    Tstar = np.empty((12,6))
    
    print(ts.data_vars)
    
    for im,m in enumerate(range(12)):
        for ireg,reg in enumerate(ts.data_vars):
            Tstar[im,ireg] = compute_Tstar(ts[reg].values[m::12],ts[reg].values[m::12])
            
    if return_arr:
        return Tstar,Tstar.min(axis=0),Tstar.min()
    if not return_arr:
        return Tstar.min(axis=0),Tstar.min()

In [5]:
compute_PM_ctrl_run_Tstar('NPP', return_arr=True)

Data variables:
    SouthernOcean  (month) float64 ...
    Weddell        (month) float64 ...
    Indian         (month) float64 ...
    WestPacific    (month) float64 ...
    Ross           (month) float64 ...
    AmundBell      (month) float64 ...


(array([[195., 150., 263., 285., 227., 255.],
        [188., 194., 277., 289., 223., 207.],
        [201., 182., 271., 255., 207., 242.],
        [256., 233., 274., 272., 283., 274.],
        [283., 270., 293., 261., 268., 280.],
        [285., 268., 274., 288., 282., 281.],
        [267., 270., 280., 291., 253., 284.],
        [266., 264., 270., 281., 279., 256.],
        [193., 141., 228., 251., 269., 274.],
        [176.,  78., 195., 228., 264., 278.],
        [ 78.,  53., 222., 145., 136., 200.],
        [131.,  87., 267., 281., 199., 197.]]),
 array([ 78.,  53., 195., 145., 136., 197.]),
 53.0)

In [6]:
compute_PM_ctrl_run_Tstar('SIE', return_arr=True)

Data variables:
    SouthernOcean  (month) float64 ...
    Weddell        (month) float64 ...
    Indian         (month) float64 ...
    WestPacific    (month) float64 ...
    Ross           (month) float64 ...
    AmundBell      (month) float64 ...


(array([[226., 169., 285., 284., 255., 271.],
        [243., 228., 275., 287., 265., 250.],
        [209., 207., 292., 272., 251., 278.],
        [208., 141., 286., 280., 226., 275.],
        [210., 118., 291., 280., 247., 246.],
        [156.,  78., 276., 260., 227., 240.],
        [102.,  56., 254., 209., 202., 212.],
        [ 64.,  48., 201., 135., 190., 196.],
        [ 50.,  43., 145., 102., 187., 159.],
        [ 48.,  42., 130.,  78., 213., 161.],
        [ 63.,  49., 163.,  96., 255., 170.],
        [111.,  61., 212., 220., 230., 140.]]),
 array([ 48.,  42., 130.,  78., 187., 140.]),
 42.0)

In [7]:
compute_PM_ctrl_run_Tstar('SFC_IRR', return_arr=True)

Data variables:
    SouthernOcean  (month) float64 ...
    Weddell        (month) float64 ...
    Indian         (month) float64 ...
    WestPacific    (month) float64 ...
    Ross           (month) float64 ...
    AmundBell      (month) float64 ...


(array([[272., 240., 280., 283., 266., 292.],
        [276., 283., 283., 277., 280., 271.],
        [293., 264., 274., 275., 273., 287.],
        [281., 278., 284., 275., 282., 281.],
        [274., 271., 289., 265., 291., 263.],
        [281., 268., 273., 282., 279., 280.],
        [221., 151., 251., 275., 282., 278.],
        [100.,  60., 212., 184., 240., 275.],
        [ 68.,  48., 148., 180., 235., 260.],
        [ 81.,  52., 158., 162., 203., 246.],
        [101.,  79., 275., 185., 254., 241.],
        [205., 151., 278., 283., 276., 267.]]),
 array([ 68.,  48., 148., 162., 203., 241.]),
 48.0)

In [8]:
compute_PM_ctrl_run_Tstar('MLD', return_arr=True)

Data variables:
    SouthernOcean  (month) float64 ...
    Weddell        (month) float64 ...
    Indian         (month) float64 ...
    WestPacific    (month) float64 ...
    Ross           (month) float64 ...
    AmundBell      (month) float64 ...


(array([[283., 293., 270., 288., 287., 282.],
        [274., 291., 267., 266., 271., 265.],
        [279., 288., 277., 274., 285., 289.],
        [283., 285., 282., 283., 284., 292.],
        [290., 258., 264., 288., 288., 282.],
        [183., 164., 256., 268., 256., 265.],
        [ 71.,  72., 153., 266., 224., 229.],
        [ 54.,  57.,  92., 239., 206., 205.],
        [ 58.,  66.,  62., 212., 226., 255.],
        [ 81.,  89.,  62., 269., 239., 275.],
        [261., 250., 261., 287., 286., 290.],
        [285., 283., 294., 279., 287., 290.]]),
 array([ 54.,  57.,  62., 212., 206., 205.]),
 54.0)

In [9]:
compute_PM_ctrl_run_Tstar('SFC_CHL', return_arr=True)

Data variables:
    SouthernOcean  (month) float64 ...
    Weddell        (month) float64 ...
    Indian         (month) float64 ...
    WestPacific    (month) float64 ...
    Ross           (month) float64 ...
    AmundBell      (month) float64 ...


(array([[248., 158., 265., 274., 241., 294.],
        [275., 250., 257., 283., 267., 287.],
        [273., 269., 257., 268., 260., 290.],
        [265., 292., 279., 270., 284., 279.],
        [280., 240., 248., 256., 285., 268.],
        [270., 196., 284., 260., 255., 271.],
        [257., 168., 269., 279., 244., 270.],
        [252., 124., 215., 282., 275., 266.],
        [260., 135., 213., 274., 274., 283.],
        [270., 274., 218., 285., 263., 275.],
        [276., 241., 284., 277., 277., 276.],
        [250., 118., 244., 269., 279., 281.]]),
 array([248., 118., 213., 256., 241., 266.]),
 118.0)

In [10]:
compute_PM_ctrl_run_Tstar('SFC_BIOMASS', return_arr=True)

Data variables:
    SouthernOcean  (month) float64 ...
    Weddell        (month) float64 ...
    Indian         (month) float64 ...
    WestPacific    (month) float64 ...
    Ross           (month) float64 ...
    AmundBell      (month) float64 ...


(array([[241., 122., 243., 275., 232., 280.],
        [252., 105., 255., 287., 221., 276.],
        [258., 174., 259., 288., 280., 249.],
        [278., 263., 265., 283., 287., 280.],
        [287., 263., 269., 257., 282., 274.],
        [274., 222., 285., 266., 258., 274.],
        [266., 201., 269., 285., 252., 277.],
        [275., 208., 244., 283., 289., 264.],
        [269., 281., 256., 269., 272., 280.],
        [274., 266., 277., 282., 270., 276.],
        [234., 128., 269., 261., 273., 275.],
        [290., 233., 268., 280., 283., 267.]]),
 array([234., 105., 243., 257., 221., 249.]),
 105.0)

### Computing the $F$-test critical value at different confidence levels

`scipy.stats.f.ppf(q, dfn, dfd)` is the inverse CDF (i.e., the quantile function) which we can use to generate the critical values for an $F$ test with `dfn` as the DOF in the numerator and `dfd` as the DOF in the denominator.

[scipy.stats.rv_continuous.ppf](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.rv_continuous.ppf.html#scipy.stats.rv_continuous.ppf) - Percent point function (inverse of cdf) at $q$ of the given RV

### Performing the $F$ test
[F Statistic/F value](https://www.statisticshowto.com/probability-and-statistics/f-statistic-value-test/#WhatisF)

[F Test](https://www.statisticshowto.com/probability-and-statistics/hypothesis-testing/f-test/)

## Correlation
Here, we calculate the effective DOFs ($T^*$) and then use a $t$-test to assess significance.
$$
T_{XY}^*=\frac{T}{\sum\limits_{\tau=-(T-1)}^{(T-1)} (1-|\tau|/T)\rho_\tau^X\rho_\tau^Y}
$$

Formula for converting correlation coefficient $r$ to $t$-test value:
$$
\text{t-value}=r\sqrt{\frac{T^*}{1-r^2}}
$$

### Compute $T^*$ for correlation

In [11]:
def compute_corr_Tstar(predictor, predictand, maxlag, return_arr=False):
    ## convert DataArray to NumPy array
    predictor = predictor.values
    predictand = predictand.values
    
    assert predictor.size == predictand.size, 'predictor and predictand have different lengths'
    N = predictor.size

    ## rows are different init months, cols are different lags
    Tstar_matrix = np.zeros((abs(maxlag)+1, 12))

    init_matrix = np.zeros((abs(maxlag)+1, 12))
    lag_matrix = np.zeros((abs(maxlag)+1, 12))

    lag_values = np.arange(maxlag, 1)
    for (it,init) in enumerate(range(0,12)):
        for (ig,lag) in enumerate(lag_values):   
            trim = 12*((abs(lag)-1)//12+1)
            init_matrix[ig][it] = init
            lag_matrix[ig][it] = lag

            if lag == 0:
                tmp_predictand = predictand[init:N:12]
                tmp_predictor = predictor[init:N:12]

            else:
                tmp_predictand = predictand[init+trim:N:12]
                tmp_predictor = predictor[init+trim+lag:N-trim+init:12]

            Tstar_matrix[ig][it] = compute_Tstar(tmp_predictand, tmp_predictor, T=min([len(tmp_predictand), len(tmp_predictor)]))
            
    min_Tstar = Tstar_matrix.min()
        
    if return_arr:
        return Tstar_matrix, min_Tstar
    if not return_arr:
        return min_Tstar

### SIE predicting NPP

In [14]:
predictor = 'SIE'
predictand = 'NPP'
maxlag = -24
region_list = ['Weddell', 'Indian', 'WestPacific', 'SouthernOcean', 'Ross', 'AmundBell']   
sie_pred_npp_Tstar = np.zeros(6)

print('predictor =',predictor)
print('predictand =',predictand)
print()
for (ireg,reg) in enumerate(region_list):  
    ts_predictor = open_metric(predictor, 'so', 'anom')[reg]
    ts_predictand = open_metric(predictand, 'so', 'anom')[reg]
    min_Tstar = compute_corr_Tstar(ts_predictor, ts_predictand, maxlag=maxlag)
    print(reg,' --> T* = {t:.1f}'.format(t=min_Tstar))
    sie_pred_npp_Tstar[ireg] = min_Tstar

predictor = SIE
predictand = NPP

Weddell  --> T* = 47.0
Indian  --> T* = 166.0
WestPacific  --> T* = 112.0
SouthernOcean  --> T* = 62.0
Ross  --> T* = 166.0
AmundBell  --> T* = 176.0


In [16]:
sie_pred_npp_Tstar

array([ 47., 166., 112.,  62., 166., 176.])

In [20]:
region_list = ['Weddell', 'Indian', 'WestPacific', 'SouthernOcean', 'Ross', 'AmundBell'] 
sie_pred_npp_tcrit = np.zeros(6)

for it,t in enumerate(sie_pred_npp_Tstar):
    for tcrit in np.linspace(1,3,1000):
        if np.round(stats.t.cdf(tcrit,t), 3) == 0.975:
            print(region_list[it])
            print('T* = {a:.1f}'.format(a=t))
            print('critical t-value = {a:.4f}\n'.format(a=tcrit))
            sie_pred_npp_tcrit[it] = tcrit
            break

Weddell
T* = 47.0
critical t-value = 2.0030

Indian
T* = 166.0
critical t-value = 1.9670

WestPacific
T* = 112.0
critical t-value = 1.9730

SouthernOcean
T* = 62.0
critical t-value = 1.9910

Ross
T* = 166.0
critical t-value = 1.9670

AmundBell
T* = 176.0
critical t-value = 1.9650



In [24]:
region_list = ['Weddell', 'Indian', 'WestPacific', 'SouthernOcean', 'Ross', 'AmundBell'] 
sie_pred_npp_rthresh = np.zeros(6)

for i,tcrit in enumerate(sie_pred_npp_tcrit):
    for r in np.linspace(0,1,1000):
        if np.round(r * np.sqrt(sie_pred_npp_Tstar[i] / (1 - r**2)), 2) == np.round(tcrit, 2):
            print(region_list[i])
            print('T* = {a:.1f}'.format(a=sie_pred_npp_Tstar[i]))
            print('critical t-value = {a:.4f}'.format(a=tcrit))
            print('corr coeff threshold = {a:.4f}\n'.format(a=r))
            sie_pred_npp_rthresh[i] = r
            break

Weddell
T* = 47.0
critical t-value = 2.0030
corr coeff threshold = 0.2803

Indian
T* = 166.0
critical t-value = 1.9670
corr coeff threshold = 0.1512

WestPacific
T* = 112.0
critical t-value = 1.9730
corr coeff threshold = 0.1832

SouthernOcean
T* = 62.0
critical t-value = 1.9910
corr coeff threshold = 0.2452

Ross
T* = 166.0
critical t-value = 1.9670
corr coeff threshold = 0.1512

AmundBell
T* = 176.0
critical t-value = 1.9650
corr coeff threshold = 0.1461



In [29]:
sie_pred_npp_rthresh

array([0.28028028, 0.15115115, 0.18318318, 0.24524525, 0.15115115,
       0.14614615])

### SFC_IRR predicting NPP

In [15]:
predictor = 'SFC_IRR'
predictand = 'NPP'
maxlag = -24
region_list = ['Weddell', 'Indian', 'WestPacific', 'SouthernOcean', 'Ross', 'AmundBell']
irr_pred_npp_Tstar = np.zeros(6)

print('predictor =',predictor)
print('predictand =',predictand)
print()
for (ireg,reg) in enumerate(region_list):  
    ts_predictor = open_metric(predictor, 'so', 'anom')[reg]
    ts_predictand = open_metric(predictand, 'so', 'anom')[reg]
    min_Tstar = compute_corr_Tstar(ts_predictor, ts_predictand, maxlag=maxlag)
    print(reg,' --> T* = {t:.1f}'.format(t=min_Tstar))
    irr_pred_npp_Tstar[ireg] = min_Tstar

predictor = SFC_IRR
predictand = NPP

Weddell  --> T* = 51.0
Indian  --> T* = 173.0
WestPacific  --> T* = 156.0
SouthernOcean  --> T* = 73.0
Ross  --> T* = 170.0
AmundBell  --> T* = 227.0


In [17]:
irr_pred_npp_Tstar

array([ 51., 173., 156.,  73., 170., 227.])

In [27]:
region_list = ['Weddell', 'Indian', 'WestPacific', 'SouthernOcean', 'Ross', 'AmundBell'] 
irr_pred_npp_tcrit = np.zeros(6)

for it,t in enumerate(irr_pred_npp_Tstar):
    for tcrit in np.linspace(1,3,1000):
        if np.round(stats.t.cdf(tcrit,t), 3) == 0.975:
            print(region_list[it])
            print('T* = {a:.1f}'.format(a=t))
            print('critical t-value = {a:.4f}\n'.format(a=tcrit))
            irr_pred_npp_tcrit[it] = tcrit
            break

Weddell
T* = 51.0
critical t-value = 1.9990

Indian
T* = 173.0
critical t-value = 1.9670

WestPacific
T* = 156.0
critical t-value = 1.9670

SouthernOcean
T* = 73.0
critical t-value = 1.9850

Ross
T* = 170.0
critical t-value = 1.9670

AmundBell
T* = 227.0
critical t-value = 1.9630



In [28]:
region_list = ['Weddell', 'Indian', 'WestPacific', 'SouthernOcean', 'Ross', 'AmundBell'] 
irr_pred_npp_rthresh = np.zeros(6)

for i,tcrit in enumerate(irr_pred_npp_tcrit):
    for r in np.linspace(0,1,1000):
        if np.round(r * np.sqrt(irr_pred_npp_Tstar[i] / (1 - r**2)), 2) == np.round(tcrit, 2):
            print(region_list[i])
            print('T* = {a:.1f}'.format(a=irr_pred_npp_Tstar[i]))
            print('critical t-value = {a:.4f}'.format(a=tcrit))
            print('corr coeff threshold = {a:.4f}\n'.format(a=r))
            irr_pred_npp_rthresh[i] = r
            break

Weddell
T* = 51.0
critical t-value = 1.9990
corr coeff threshold = 0.2693

Indian
T* = 173.0
critical t-value = 1.9670
corr coeff threshold = 0.1481

WestPacific
T* = 156.0
critical t-value = 1.9670
corr coeff threshold = 0.1562

SouthernOcean
T* = 73.0
critical t-value = 1.9850
corr coeff threshold = 0.2252

Ross
T* = 170.0
critical t-value = 1.9670
corr coeff threshold = 0.1491

AmundBell
T* = 227.0
critical t-value = 1.9630
corr coeff threshold = 0.1291



In [30]:
irr_pred_npp_rthresh

array([0.26926927, 0.14814815, 0.15615616, 0.22522523, 0.14914915,
       0.12912913])

---

In [7]:
def compute_t_stat(predictor, predictand):
    ## convert DataArray to NumPy array
    predictor = predictor.values
    predictand = predictand.values
    
    ## rows are different init months, cols are different lags
    eff_dof = np.zeros((abs(maxlag)+1, 12))
    
    for (it,init) in enumerate(range(0,12)):
        for (ig,lag) in enumerate(range(0,12)):
            
            tmp_predictand = predictand[init:N:12]
            tmp_predictor = predictor[init:N:12]
            
            

In [14]:
def open_metric(var, reg, metric, timescale='monthly', ens_type=''):
    
    writedir = '/home/bbuchovecky/storage/so_predict_derived/'
    
    if metric == 'clim':
        subdir = 'CTRL/'+var.upper()+'/'
        filename = var.lower()+'_ts_'+reg+'_'+metric+'.nc'
    
    if metric == 'anom' or metric == 'mean' or (metric == 'var' and timescale == 'monthly'):
        subdir = 'CTRL/'+var.upper()+'/'
        filename = var.lower()+'_ts_'+reg+'_'+timescale+'_'+metric+'.nc'
    
    if metric.lower() == 'ppp':
        subdir = 'PPP/'+var.upper()+'/'
        if ens_type != '':
            ens_type += '_'
        filename = var.lower()+'_ts_'+reg+'_'+timescale+'_'+ens_type+'ppp.nc'
        
    return xr.open_dataset(writedir+subdir+filename)

def get_plotting_labels():
    with open('/home/bbuchovecky/storage/so_predict_derived/plotting_dicts.pkl','rb') as handle:
        plotting_dicts = pkl.load(handle)
    
    reg_names = plotting_dicts['reg_names']
    var_abbrv_names = plotting_dicts['var_abbrv_names']
    abbrv_month_names = plotting_dicts['abbrv_month_names']
    month_letters = plotting_dicts['month_letters']
    var_su_names = plotting_dicts['var_su_names']
    
    return reg_names, var_abbrv_names, abbrv_month_names, month_letters, var_su_names

In [15]:
## predictor --> the field doing the predicting (e.g., SIE or SFC_IRR)
## predictand --> the field being predicted (e.g., NPP)
def compute_lagged_xcorr(predictor, predictand, maxlag):
    ## convert DataArray to NumPy array
    predictor = predictor.values
    predictand = predictand.values
    
    assert predictor.size == predictand.size, 'predictor and predictand have different lengths'
    N = predictor.size
    
    ## rows are different init months, cols are different lags
    r_matrix = np.zeros((abs(maxlag)+1, 12))
    p_matrix = np.zeros((abs(maxlag)+1, 12))

    init_matrix = np.zeros((abs(maxlag)+1, 12))
    lag_matrix = np.zeros((abs(maxlag)+1, 12))
    
    lag_values = np.arange(maxlag, 1)
    for (it,init) in enumerate(range(0,12)):
        for (ig,lag) in enumerate(lag_values):   
            trim = 12*((abs(lag)-1)//12+1)
            init_matrix[ig][it] = init
            lag_matrix[ig][it] = lag
            
            if lag == 0:
                tmp_predictand = predictand[init:N:12]
                tmp_predictor = predictor[init:N:12]
                
            else:
                tmp_predictand = predictand[init+trim:N:12]
                tmp_predictor = predictor[init+trim+lag:N-trim+init:12]
            
            r_matrix[ig][it], p_matrix[ig][it] = stats.pearsonr(tmp_predictand, tmp_predictor)
            
    return r_matrix, p_matrix, init_matrix, lag_matrix